# Mohammad Ali Mojtahed Soleimani

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'bert-base-uncased'
num_epochs = 10
batch_size = 16
learning_rate = 2e-5
save_dir = './saved_models'
data_path = './IMDB.csv'

In [16]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [17]:
class IMDBDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_len):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = IMDBDataset(
        reviews=df.review.to_numpy(),
        labels=df.sentiment.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size, num_workers=2)

In [18]:
df = pd.read_csv(data_path)

# --- Convert sentiment labels to numerical values (if needed) ---
if df['sentiment'].dtype == 'object':
    label_map = {'positive': 1, 'negative': 0}  # Customize if your labels are different
    df['sentiment'] = df['sentiment'].map(label_map)


In [19]:
train_df = df.sample(frac=0.8, random_state=42)  # 80% for training
test_df = df.drop(train_df.index)  # Remaining 20% for testing

# --- Tokenizer and Data Loaders ---
tokenizer = BertTokenizer.from_pretrained(model_name)
MAX_LEN = 512

train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, batch_size)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, batch_size)

# --- Model ---
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = model.to(device)

# --- Optimizer and Loss Function ---
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss().to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# --- Training Loop ---
train_losses = []
test_losses = []
train_f1_scores = []
test_f1_scores = []

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('-' * 10)

    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    train_preds = []
    train_targets = []

    for batch in tqdm(train_data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.logits, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        train_preds.extend(predicted.cpu().numpy())
        train_targets.extend(labels.cpu().numpy())

    avg_train_loss = train_loss / len(train_data_loader)
    train_losses.append(avg_train_loss)

    train_f1 = f1_score(train_targets, train_preds)
    train_f1_scores.append(train_f1)

    # --- Evaluation ---
    model.eval()
    test_loss = 0
    test_correct = 0
    test_total = 0
    test_preds = []
    test_targets = []

    with torch.no_grad():
        for batch in tqdm(test_data_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

            test_preds.extend(predicted.cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    avg_test_loss = test_loss / len(test_data_loader)
    test_losses.append(avg_test_loss)

    test_f1 = f1_score(test_targets, test_preds)
    test_f1_scores.append(test_f1)

    print(f'Train Loss: {avg_train_loss:.4f}, Train F1: {train_f1:.4f}')
    print(f'Test Loss: {avg_test_loss:.4f}, Test F1: {test_f1:.4f}')

    # --- Save model after each epoch ---
    torch.save(model.state_dict(), os.path.join(save_dir, f'model_epoch_{epoch+1}.pth'))


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_f1_scores, label='Train F1 Score')
plt.plot(test_f1_scores, label='Test F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Training and Test F1 Score')
plt.legend()

plt.tight_layout()
plt.show()